<div>
    <h1 style="margin-top: 50px; font-size: 33px; text-align: center"> Homework 4 </h1>
    <br>
    <div style="font-weight:200; font-size: 20px; padding-bottom: 15px; width: 100%; text-align: center;">
        <right>Fabio Montello, Maria Luisa Croci, Eltaj Babanli</right>
        <br>
    </div>
    <hr>
</div>

<div>
    <h1 style="margin-top: -5px; font-size: 20px; text-align: center"> 2) Find the duplicates! </h1>
    <br>
</div>

Starting from passwords2.txt file as input with each row having a string of 20 characters we want to define a hash function that associates a value to each string. 
The goal is to *check whether there are some duplicate strings*.

### Part 1

For the first part we define duplicate when two strings have the same characters, order is not important. Thus, "AABA" = "AAAB", and we count it as *one duplicate*. 

Since the amount of data is very large, we decided to use Hadoop Spark to process all the data. This choice will let us use the full potential on the CPU enabling the use of multicore and increasing the speed of execution. It will also reduce the amount of RAM needed for the esecution.
We start by importing the `pyspark` environment, which is an interface to use Spark on python:

In [1]:
import findspark
import pyspark

In [2]:
findspark.init()
sc = pyspark.SparkContext(appName="findDuplicates")

In [3]:
sc

<SparkContext master=local[*] appName=findDuplicates>

Then we proceed by telling to the Spark environment which file text take as input. 

In [4]:
txt = sc.textFile("data/passwords2.txt")

Let's not see if the data gets properly imported by printing the first 5 number:

In [5]:
txt.take(5)

['OHcv-/U3QI$rdqYTef"D',
 'QtA*.xM$e(+"aO36r&Uo',
 "T;rqw/ou'HN-Pklj6hM*",
 'b%xJ79"A*C5@ehMfS3lu',
 'buI=;LpjBiCm"JS5\'#xy']

At this point we want to define the function that generates the hash function from the string. A hash function takes an item of a given type and generates an integer hash value within a given range. The input items can be anything: strings, compiled shader programs, files, even directories. The same input always generates the same hash value, and a good hash function tends to generate different hash values when given different inputs. A hash function has no awareness of “other” items in the set of inputs. It just performs some arithmetic and/or bit-magic operations on the input item passed to it. Therefore, there’s always a chance that two different inputs will generate the same hash value [[1]](https://preshing.com/20110504/hash-collision-probabilities/).

Now let's build the function that will generate the first hash function. Since we do not need to encrypt the data, but just count the duplicates, we will not focus on encoding our hash, but just generating a reasonable code that won't take too much computational time or space in memory. In any case we want to mention that at this [link](http://www.metamorphosite.com/one-way-hash-encryption-sha1-data-software) there is an interesting example on how to use the XOR operator to make an hash function encrypted (which is the real purpose of hash functions).

Our hash function is so composed: we extract the ASCII code of every single element in the string and we sum the power to the cube of this number with all the number computed previously. 
In this way we use the commutative property to not consider the actual position of the elements. Furthermore we power the number to the cube to improve the differentation between numbers and try to avoid as much as possible the collision.

In [2]:
def hashingMap(item):
    s = 0
    for c in item:
        s += (ord(c)**3)
    return (s, 1)

It's time to compute the map and reduce functions. The reduce function is written inline, since it consists of just a counter of how many times the elements with the same keys repeats.

In [7]:
ntuples = txt.map(hashingMap).reduceByKey(lambda a, b: a+b)

In [8]:
ntuples.take(5)

[(14206962, 17), (9652134, 14), (13829394, 22), (13533246, 20), (10688514, 15)]

In [11]:
reduced = ntuples

We want to filter now all the elements that does not have duplicates:

In [3]:
duplicates = reduced.filter(lambda x: x[1]>1)

NameError: name 'reduced' is not defined

In [13]:
duplicates.take(10)

[(14206962, 17),
 (9652134, 14),
 (13829394, 22),
 (13533246, 20),
 (10688514, 15),
 (9515031, 17),
 (11009847, 24),
 (14349378, 13),
 (12953232, 23),
 (12682890, 22)]

And finally we want to count the number of duplicates:

In [14]:
n = duplicates.count()
n

9448196

We can see that we got 9448196 duplicates, which is very close to the 10000000 duplicates expected. Increasing the power to the 4 or 5 would for sure improve the quality of the algorithm avoiding collision better. At the same time it would slow down the code. That's why we think that the power to the cube is the right threshold in between efficiency and collision probability.

### Part 2

For the second part we define duplicate when two strings have the same characters and order is also important. Thus, "AABA" is not equals "AAAB".

We want to generate a function where position matters. So we decided to elaborate the data in a peculiar way. We started by composing a string that has the binary code of the whole password we have to transform. Then we wanted to unify the strings, so we added zeros where the string was not long enough, and removed characters when it was too long. In this way we controlled the length of the int variable we are going to output. In our case we decided to keep 64 bit as length of our integer, a reasonable value to not take too much space in memory.

In [15]:
def hashingMapUnique(item):
    s = ''
    for c in item:
        s += "{0:b}".format(ord(c))
    return (int(s.ljust(64, '0')[:64:],2), 1)

Again we do the map-reduce procedure to count the data, as we did previously, filtering just the duplicates and counting them.

In [16]:
ntuples = txt.map(hashingMapUnique).reduceByKey(lambda a, b: a+b)

In [17]:
ntuples.take(5)

[(16274527251664803781, 2),
 (10608099667156110106, 2),
 (16668620600776809073, 2),
 (10081794263021956144, 2),
 (13073833392952119731, 2)]

In [18]:
reduced = ntuples

In [19]:
duplicates = reduced.filter(lambda x: x[1]>1)

In [20]:
duplicates.take(10)

[(16274527251664803781, 2),
 (10608099667156110106, 2),
 (16668620600776809073, 2),
 (10081794263021956144, 2),
 (13073833392952119731, 2),
 (10014301253483687536, 2),
 (12539534125667135270, 2),
 (17272142619530314309, 2),
 (13455788835654306926, 2),
 (15110295247321585083, 2)]

In [21]:
n = duplicates.count()
n

5000000

As result we can see that the number of duplicates in precisely 5000000. This means that the choice of taking 64 bits was reasonable to keep the data not too large and at the same time have a very good precision on the counting, avoiding at all the hash collision.